# LSTM Stock Predictor Using Fear and Greed Index

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin fear and greed index values to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [16]:
import numpy as np
import pandas as pd
import hvplot.pandas

%matplotlib inline

In [17]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [18]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [19]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [20]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [21]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [22]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [23]:
# Predict Closing Prices using a 10 day window of fear and greed index values and a target of the 11th day closing price
# Try a window size anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 0 is the `Value` column
feature_column = 0
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
# Use 70% of the data for training and the remainder for testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=78, test_size=.3
)

In [26]:
# Use MinMaxScaler to scale the data between 0 and 1. 
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit the MinMaxScaler object with the features data X
scaler.fit(X)

# Scale the features training and testing sets
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Fit the MinMaxScaler object with the target data Y
scaler.fit(y)

# Scale the target training and testing sets
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [27]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [28]:
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.36781609]
  [0.3908046 ]
  [0.29885057]
  [0.3908046 ]
  [0.3908046 ]
  [0.36781609]
  [0.12790698]
  [0.38372093]
  [0.1744186 ]
  [0.44186047]]

 [[0.50574713]
  [0.54022989]
  [0.55172414]
  [0.63218391]
  [0.67816092]
  [0.55172414]
  [0.61627907]
  [0.51162791]
  [0.62790698]
  [0.37209302]]

 [[0.18390805]
  [0.2183908 ]
  [0.20689655]
  [0.12643678]
  [0.14942529]
  [0.11494253]
  [0.11627907]
  [0.15116279]
  [0.11627907]
  [0.10465116]]

 [[0.33333333]
  [0.33333333]
  [0.3908046 ]
  [0.3908046 ]
  [0.33333333]
  [0.29885057]
  [0.30232558]
  [0.27906977]
  [0.31395349]
  [0.23255814]]

 [[0.63218391]
  [0.67816092]
  [0.55172414]
  [0.62068966]
  [0.51724138]
  [0.63218391]
  [0.37209302]
  [0.40697674]
  [0.36046512]
  [0.36046512]]] 

X_test sample values:
[[[0.6091954 ]
  [0.5862069 ]
  [0.65517241]
  [0.70114943]
  [0.65517241]
  [0.64367816]
  [0.61627907]
  [0.65116279]
  [0.38372093]
  [0.61627907]]

 [[0.10344828]
  [0.11494253]
  [0.057471

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [30]:
model = Sequential()

# Initial model setup
number_units = 10
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))

In [31]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 10)            480       
_________________________________________________________________
dropout (Dropout)            (None, 10, 10)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 10)            840       
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                840       
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1

In [33]:
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=90, verbose=1)

Epoch 1/10
5/5 [==============================] - 0s 16ms/step - loss: 0.1754
Epoch 2/10
5/5 [==============================] - 0s 15ms/step - loss: 0.1359
Epoch 3/10
5/5 [==============================] - 0s 13ms/step - loss: 0.1044
Epoch 4/10
5/5 [==============================] - 0s 13ms/step - loss: 0.0793
Epoch 5/10
5/5 [==============================] - 0s 13ms/step - loss: 0.0602
Epoch 6/10
5/5 [==============================] - 0s 12ms/step - loss: 0.0557
Epoch 7/10
5/5 [==============================] - 0s 12ms/step - loss: 0.0595
Epoch 8/10
5/5 [==============================] - 0s 13ms/step - loss: 0.0591
Epoch 9/10
5/5 [==============================] - 0s 12ms/step - loss: 0.0565
Epoch 10/10
5/5 [==============================] - 0s 13ms/step - loss: 0.0526


---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [36]:
model.evaluate(X_test, y_test, verbose=1)

5/5 [==============================] - 0s 4ms/step - loss: 0.0455


0.045453544706106186

In [37]:
# Make some predictions
predicted = model.predict(X_test)

In [38]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [39]:
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
})
stocks.head()

,Real,Predicted
0,5165.589844,7188.350586
1,6261.479980,5954.557129
2,9882.429688,6367.540039
3,7371.959961,7406.785156
4,6762.060059,6066.826172


In [40]:
# Plot the real vs predicted values as a line chart
stocks.hvplot(
                y = ["Real", "Predicted"],
                width=700, height=400,
                title="Real v Predicted via Sentiment Data")

:NdOverlay   [Variable]
   :Curve   [index]   (value)